# Part 1

Write a function that estimates the emission parameters from the training set using MLE (maximum
likelihood estimation):

In [1]:
def read_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        file_content = file.read()
    return file_content

def write_file(file_path, content):
    with open(file_path, 'w', encoding="utf-8") as file:
        file.write(content)

In [2]:
def estimate_emission_parameters(file_content):
    tags = {}  # Dictionary to store counts of each observation
    emission_params = {}  # Dictionary to store emission probabilities

    # Split the file content into lines
    lines = file_content.strip().split("\n")

    # Iterate through each line and extract observations and tags
    for line in lines:
        if len(line) == 0:
            continue
        if line[-1] == 'O':
            observation, tag = line[:-1], line[-1]
        elif ('B-positive' in line 
            or 'I-positive' in line 
            or 'B-negative' in line 
            or 'I-negative' in line):
            observation, tag = line[:-10], line[-10:]
        elif ('B-neutral' in line
            or 'I-neutral' in line):
            observation, tag = line[:-9], line[-9:]
#         observation, tag = line.split()
    
        if tag in tags:
            tags[tag][observation] = tags[tag].get(observation, 0) + 1
        else:
            tags[tag] = {observation: 1}

    # Calculate emission probabilities for each observation and tag
    for tag, observations_count in tags.items():
        total_count = sum(observations_count.values())
        emission_params[tag] = {
            observation: count / total_count for observation, count in observations_count.items()
        }

    return emission_params

file_content = read_file("ES/train.txt")

emission_parameters = estimate_emission_parameters(file_content)
# print(emission_parameters, sum(len(words) for words in emission_parameters.values()))

One problem with estimating the emission parameters is that some words that appear in the test set
do not appear in the training set. One simple idea to handle this issue is as follows. We introduce
a special word token #UNK#, and make the following modifications to the computation of emission
probabilities.During the testing phase, if the word does not appear in the training set, we replace that word with
#UNK#. Set k to 1, implement this fix into your function for computing the emission parameters.

In [3]:
def estimate_emission_parameters_modified(training_file_content, test_file_content, k=1):
    tags = {}  # Dictionary to store counts of each observation
    emission_params = {}  # Dictionary to store emission probabilities
    train_words = []
    test_words = []
    
    # Split the file content into lines
    train_data_lines = training_file_content.strip().split("\n")
    test_data_lines = test_file_content.strip().split("\n")

    # Iterate through each line and extract observations and tags
    for line in train_data_lines:
        if len(line) == 0:
            continue
        if ' O' in line:
            observation, tag = line[:-2], line[-1]
        elif ('B-positive' in line 
            or 'I-positive' in line 
            or 'B-negative' in line 
            or 'I-negative' in line):
            observation, tag = line[:-11], line[-10:]
        elif ('B-neutral' in line
            or 'I-neutral' in line):
            observation, tag = line[:-10], line[-9:]
        if observation not in train_words:
            train_words.append(observation)
        if tag in tags:
            tags[tag][observation] = tags[tag].get(observation, 0) + 1
        else:
            tags[tag] = {observation: 1}
    # Iterate through each line to extract observations from test set
    for line in test_data_lines:
        if len(line) == 0:
            continue
        if ' O' in line:
            observation, tag = line[:-2], line[-1]
        elif ('B-positive' in line 
            or 'I-positive' in line 
            or 'B-negative' in line 
            or 'I-negative' in line):
            observation, tag = line[:-11], line[-10:]
        elif ('B-neutral' in line
            or 'I-neutral' in line):
            observation, tag = line[:-10], line[-9:]
        else:
            observation = line
        if observation not in test_words:
            test_words.append(observation)
    # Extract words that are in test set but not in train set   
    unique_words = find_unique_words(test_words, train_words)

    # Calculate emission probabilities for each observation and tag
    for tag, observations_count in tags.items():
        total_count = sum(observations_count.values())
        emission_params[tag] = {
            observation: count / (total_count + k) for observation, count in observations_count.items()
        }
        for word in unique_words:
            emission_params[tag][word] = k / (total_count + k)


    return emission_params, list(set(test_words).union(train_words))

def find_unique_words(list1, list2):
    set1 = set(list1)
    set2 = set(list2)

    unique_in_list1 = set1 - set2

    return list(unique_in_list1)


training_file_content = read_file("ES/train.txt")
test_file_content = read_file("ES/dev.in")
emission_parameters = estimate_emission_parameters_modified(training_file_content, test_file_content, 1)[0]
# print(emission_parameters, sum(len(words) for words in emission_parameters.values()))
count = 0
for emission_probabilities in emission_parameters.values():
    count += len(emission_probabilities)
# print(count)

In [4]:
def sentiment_analysis(path_to_train_file, path_to_test_file):
    training_file_content = read_file(path_to_train_file)
    test_file_content = read_file(path_to_test_file)
    emission_parameters, words = estimate_emission_parameters_modified(training_file_content, test_file_content)
    word_to_label = {}
    for word in words:
        probabilities = []
        for label, freqs in emission_parameters.items():
            if word in freqs:
                probabilities.append((label, freqs[word]))
        word_to_label[word] = max(probabilities, key=lambda x: x[1])[0]
    return word_to_label

def write_result_to_file(word_to_label, path_to_dev_set, path_to_output):
    result = ""
    dev_file_content = read_file(path_to_dev_set)
    dev_set_lines = dev_file_content.strip().split("\n")
    counter = 0
    for line in dev_set_lines:
        if line == "":
            result += "\n"
            continue  
        result += line + " " + word_to_label.get(line) + "\n"
 
    write_file(path_to_output, result)        

In [5]:
mapping = sentiment_analysis("ES/train.txt", "ES/dev.in")
write_result_to_file(mapping, "ES/dev.in", "ES/dev.p1.out")

In [6]:
mapping = sentiment_analysis("RU/train.txt", "RU/dev.in")
write_result_to_file(mapping, "RU/dev.in", "RU/dev.p1.out")

## ES
#Entity in gold data: 229
#Entity in prediction: 1466

#Correct Entity : 178
Entity  precision: 0.1214
Entity  recall: 0.7773
Entity  F: 0.2100

#Correct Sentiment : 97
Sentiment  precision: 0.0662
Sentiment  recall: 0.4236
Sentiment  F: 0.1145

## RU
#Entity in gold data: 389
#Entity in prediction: 1816

#Correct Entity : 266
Entity  precision: 0.1465
Entity  recall: 0.6838
Entity  F: 0.2413

#Correct Sentiment : 129
Sentiment  precision: 0.0710
Sentiment  recall: 0.3316
Sentiment  F: 0.1170

# Part 2

Write a function that estimates the transition parameters from the training set using MLE (maximum
likelihood estimation):

In [7]:
def estimate_transition_parameters(file_content):
    transition_counts = {}
    transition_params = {}  # Dictionary to store emission probabilities

    # Split the file content into lines
    lines = file_content.strip().split("\n")

    # Iterate through each line and extract observations and tags
    for line_index in range(len(lines)):
        next_line = ""
        curr_line = lines[line_index]
        if line_index < len(lines) - 1:
            next_line = lines[line_index + 1]
        
        if len(curr_line) == 0:
            continue
        if curr_line[-1] == 'O':
            curr_observation, curr_tag = curr_line[:-1], curr_line[-1]
        elif ('B-positive' in curr_line 
            or 'I-positive' in curr_line 
            or 'B-negative' in curr_line 
            or 'I-negative' in curr_line):
            curr_observation, curr_tag = curr_line[:-10], curr_line[-10:]
        elif ('B-neutral' in curr_line
            or 'I-neutral' in curr_line):
            curr_observation, curr_tag = curr_line[:-9], curr_line[-9:]
        
        if len(next_line) == 0: #true if line_index = len(lines - 1) or line_index is last index of a document
            pass
        elif next_line[-1] == 'O':
            next_observation, next_tag = next_line[:-1], next_line[-1]
        elif ('B-positive' in next_line 
            or 'I-positive' in next_line 
            or 'B-negative' in next_line 
            or 'I-negative' in next_line):
            next_observation, next_tag = next_line[:-10], next_line[-10:]
        elif ('B-neutral' in next_line
            or 'I-neutral' in next_line):
            next_observation, next_tag = next_line[:-9], next_line[-9:]        
        
        # Handle transitions from START
        if line_index == 0 or lines[line_index - 1] == "":
            if 'START' in transition_counts:
                transition_counts['START'][curr_tag] = transition_counts['START'].get(curr_tag, 0) + 1
            else:
                transition_counts['START'] = {curr_tag: 1}
                
        # Handle transitions to STOP
        if len(next_line) == 0:
            if curr_tag in transition_counts:
                transition_counts[curr_tag]['STOP'] = transition_counts[curr_tag].get('STOP', 0) + 1
            else:
                transition_counts[curr_tag] = {'STOP': 1}
                
        # Rest of the transitions
        else:
            if curr_tag in transition_counts:
                transition_counts[curr_tag][next_tag] = transition_counts[curr_tag].get(next_tag, 0) + 1
            else:
                transition_counts[curr_tag] = {next_tag: 1}
        

    # Calculate transition probabilities 
    for tag, next_tag_counts in transition_counts.items():
        total_count = sum(next_tag_counts.values())
        transition_params[tag] = {
            next_tag: count / total_count for next_tag, count in next_tag_counts.items()
        }

    return transition_params

file_content = read_file("ES/train.txt")

transition_parameters = estimate_transition_parameters(file_content)
print(transition_parameters)

{'START': {'O': 0.9289176090468497, 'B-positive': 0.052234787291330104, 'B-negative': 0.014001077005923533, 'B-neutral': 0.004846526655896607}, 'O': {'O': 0.8856896848630963, 'B-positive': 0.03650766316514551, 'STOP': 0.06344067504735663, 'B-negative': 0.012226623041157224, 'B-neutral': 0.0021353538832443605}, 'B-positive': {'O': 0.871551724137931, 'I-positive': 0.11637931034482758, 'STOP': 0.008620689655172414, 'B-neutral': 0.0008620689655172414, 'B-positive': 0.002586206896551724}, 'B-negative': {'O': 0.8110236220472441, 'I-negative': 0.1784776902887139, 'STOP': 0.010498687664041995}, 'B-neutral': {'I-neutral': 0.20833333333333334, 'O': 0.7916666666666666}, 'I-neutral': {'I-neutral': 0.6511627906976745, 'O': 0.3488372093023256}, 'I-positive': {'I-positive': 0.5700636942675159, 'O': 0.4267515923566879, 'STOP': 0.0031847133757961785}, 'I-negative': {'O': 0.39766081871345027, 'I-negative': 0.6023391812865497}}


In [8]:
def viterbi(observation_sequence, emission_params, transition_params):
    states = list(emission_params.keys())
    T = len(observation_sequence)
    N = len(states)

    # Initialization step
    viterbi_table = [{}]
    backpointer = [{}]
    for state in states:
        viterbi_table[0][state] = transition_params['START'].get(state, 0) * emission_params[state].get(observation_sequence[0], 0)
        backpointer[0][state] = None

    # Recursion step
    for t in range(1, T):
        viterbi_table.append({})
        backpointer.append({})
        for state in states:
            max_prob = max(
                viterbi_table[t - 1][prev_state] * transition_params[prev_state].get(state, 0) * emission_params[state].get(observation_sequence[t], 0)
                for prev_state in states
            )
            viterbi_table[t][state] = max_prob
            backpointer[t][state] = max(states, key=lambda prev_state: viterbi_table[t - 1][prev_state] * transition_params[prev_state].get(state, 0))

    # Termination step
    max_prob_last_state = max(viterbi_table[T - 1].values())
    best_last_state = max(states, key=lambda state: viterbi_table[T - 1][state] * transition_params[state].get('STOP', 0))

    # Backtrack to find the best sequence
    best_sequence = [best_last_state]
    prev_state = best_last_state
    for t in range(T - 2, -1, -1):
        best_sequence.insert(0, backpointer[t + 1][prev_state])
        prev_state = backpointer[t + 1][prev_state]

    return best_sequence

def create_observation_sequences(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        file_content = file.read().strip()

    observation_sequences = []
    current_sequence = []

    # Split the file content by empty lines to get individual sequences
    sequence_lines = file_content.split("\n\n")

    for sequence_line in sequence_lines:
        # Split each sequence into individual observations (words)
        observation_sequence = sequence_line.strip().split("\n")
        current_sequence.extend(observation_sequence)
        observation_sequences.append(current_sequence)
        current_sequence = []

    return observation_sequences

def viterbi_for_sequences(observation_sequences, emission_params, transition_params):
    best_sequences = []
    for observation_sequence in observation_sequences:
        best_sequence = viterbi(observation_sequence, emission_params, transition_params)
        best_sequences.append(best_sequence)

    return best_sequences

def write_sequences_to_file(output_file_path, observation_sequences, best_sequences):
    with open(output_file_path, 'w', encoding='utf-8') as file:
        for obs_sequence, best_sequence in zip(observation_sequences, best_sequences):
            for word, tag in zip(obs_sequence, best_sequence):
                file.write(f"{word} {tag}\n")
            file.write("\n")  # Separate sequences with an empty line

In [9]:
training_file_content = read_file("ES/train.txt")
test_file_content = read_file("ES/dev.in")
emission_params = estimate_emission_parameters_modified(training_file_content, test_file_content, 1)[0]
transition_params = estimate_transition_parameters(training_file_content)
observation_sequences = create_observation_sequences("ES/dev.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file("ES/dev.p2.out", observation_sequences, best_sequences)

In [10]:
training_file_content = read_file("RU/train.txt")
test_file_content = read_file("RU/dev.in")
emission_params = estimate_emission_parameters_modified(training_file_content, test_file_content, 1)[0]
transition_params = estimate_transition_parameters(training_file_content)
observation_sequences = create_observation_sequences("RU/dev.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file("RU/dev.p2.out", observation_sequences, best_sequences)

## ES
#Entity in gold data: 229
#Entity in prediction: 542

#Correct Entity : 134
Entity  precision: 0.2472
Entity  recall: 0.5852
Entity  F: 0.3476

#Correct Sentiment : 97
Sentiment  precision: 0.1790
Sentiment  recall: 0.4236
Sentiment  F: 0.2516

## RU

#Entity in gold data: 389
#Entity in prediction: 514

#Correct Entity : 190
Entity  precision: 0.3696
Entity  recall: 0.4884
Entity  F: 0.4208

#Correct Sentiment : 129
Sentiment  precision: 0.2510
Sentiment  recall: 0.3316
Sentiment  F: 0.2857

# Part 4: Design Challenge
For our design, we propose the use of Higher Order Hidden Markov Models (HOHMM). 
In a Higher Order HMM, emission pand transition probabilities are conditioned on the current state and the previous several states. This allows the model to capture longer-range dependencies in the sequence. For sentiment analysis, this could help capture more complex sentiment patterns that extend over multiple words.

## 4.1 Data Processing
Here, we aim to use integers to index each possible state in order. This is done so as to enable the use of numpy arrays for calculating of Higher-order Transition Probabilities, that take into account not just the previous state but one before it that will be done in section `4.2`. 

In [11]:
import numpy as np
import pandas as pd

In [112]:
# Functions for this step 4.1
def read_data_from_file(file_path):
    """
    Seperates txt file into texts and labels arary
    """
    texts = []
    labels = []

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 2:
                text = parts[0]
                label = parts[1]
                texts.append(text)
                labels.append(label)
    
    return texts, labels

def process_data_HOHMM(file_path):
    """
    Finds higher order transition probabilities
    INPUTS:
    - file_path (string): Path to the data file in txt format
    OUTPUTS:
    - texts: array containing all texts
    - labels: array containing coressponding labels of each text
    - states: array containing unique states
    - state_to_idx: mapping of states to a numerical index
    - label_idx: array of states in numerical idx, corresponding to texts
    
    
    """
    texts, labels = read_data_from_file(file_path)
    # Get states
    states = list(set(labels))
    
    # Define a mapping from states to indices
    state_to_idx = {state: idx for idx, state in enumerate(states)}
    
    # Get array of state indices corresponding to each text in the texts array
    label_idx = [state_to_idx[label] for label in labels]

    return texts, labels, states, state_to_idx, label_idx

In [113]:
# Data Processing for EU and RS
ES_train = "ES/train.txt"
RU_train = "RU/train.txt"

ES_texts, ES_labels, ES_states, ES_state_mappings, ES_label_idx = process_data_HOHMM(ES_train)
RU_texts, RU_labels, RU_states, RU_state_mappings, RU_label_idx = process_data_HOHMM(RU_train)

## 4.2 Calculation of Higher Order Transition Probabilities
In addition to standard HMM initialization, consider the higher order transition probabilities. These probabilities involve multiple previous states, capturing longer-range dependencies.

Simply put, we need to find the parameter $a_{u, v, q}$ which is the probability of state $q$ appearing after $u, v$ appears.
$$a_{u, v, q} = P(q|y, v) = \frac{Count(u, v, q)}{Count(u,v)}$$

whereby $Count(u, v, q)$ is the number of times `q` appears after `u, v` appears before it

In [114]:
def find_a_HOHMM(states, label_idx):
    """
    Finds a_uvq, HO transition probabilities
    INPUTS:
    - states (arr): array containing states in dataset
    - label_udx (arr): Array of Indexed Labels for each corresponding text
    OUTPUTS:
    - a_uvq (np.array): np.array containign transition probabilities
    """
    # Find number of states
    num_states = len(states)
    
    # Initiate a np array to store HO a_uvq
    transition_probs = np.zeros((num_states, num_states, num_states))
    # Finding Count(u, v, q)
    for i in range(2, len(label_idx)):
        prev_state_1 = label_idx[i - 2]
        prev_state_2 = label_idx[i - 1]
        current_state = label_idx[i]
        
        transition_probs[prev_state_1, prev_state_2, current_state] += 1
        
    # Finding HO transition params
    count_uv = np.nansum(transition_probs, axis=2, keepdims=True) 
    a_uvq = transition_probs / count_uv
    
    # Handle NaNs, convert to 0 probability
    a_uvq = np.nan_to_num(a_uvq, nan=0.0)

    return a_uvq

In [115]:
# Finding HO transition probabiltiies for ES and RU
ES_a = find_a_HOHMM(ES_states, ES_label_idx)
RU_a = find_a_HOHMM(RU_states, RU_label_idx)

/var/folders/jq/45wxrxt52wl0blhnn9xd_cb00000gn/T/ipykernel_11384/1038143059.py:24: RuntimeWarning: invalid value encountered in divide
  a_uvq = transition_probs / count_uv


## 4.3 Calculating Emission Probabilities
Follows that of normal HMM
$$b_u(o) = \frac{Count(u \to o)}{Count(u)}$$

However, we modify the function to handle np arrays. In order to take into account possible unique words, we create a vocab mapping too to prevent IndexErrors.

In [153]:
import numpy as np
import pandas as pd

def calculate_emission_probs(texts, labels, states):
    """
    Calculates emission probabilities for a Higher Order Hidden Markov Model (HOHMM).

    INPUTS:
    - texts (list): List of observed words/tokens.
    - labels (list): List of corresponding sentiment labels.
    - states (list): List of all possible states (sentiment labels).

    OUTPUTS:
    - emission_probs (np.array): Emission probabilities matrix (state x word).
    - words_to_idx (dict): Mappings from word to idx, following emissions probability np array
    """
    num_states = len(states)
    num_words = len(set(texts))
    
    # Emission probabilities (word given state)
    emission_probs = np.zeros((num_states, num_words))
    
    # Count occurrences of words for each state
    state_word_counts = np.zeros((num_states, num_words))

    # Mapping dict
    words_to_idx = {}
    
    # Process the dataset
    for i in range(len(texts)):
        state_idx = labels[i]  # Use label directly
        word = texts[i]
        word_idx = hash(word) % num_words  # Simplified hashing
        words_to_idx[word] = word_idx    
        state_word_counts[state_idx, word_idx] += 1
    
    # Calculate emission probabilities
    for state_idx in range(num_states):
        total_count = np.sum(state_word_counts[state_idx])
        emission_probs[state_idx] = state_word_counts[state_idx] / total_count
    
    return emission_probs, words_to_idx

In [154]:
RU_b, RU_wordmapping = calculate_emission_probs(RU_texts, RU_label_idx, RU_states)
ES_b, ES_wordmapping = calculate_emission_probs(ES_texts, ES_label_idx, ES_states)

## 4.4 Start Probabilities

In [147]:
def calculate_start_probs(label_indices, num_states):
    """
    Calculates start probabilities for a Higher Order Hidden Markov Model (HOHMM).

    INPUTS:
    - label_indices (list): List of label indices corresponding to each observation.
    - num_states (int): Number of possible states (sentiment labels).

    OUTPUTS:
    - start_probs (np.array): Start probabilities vector for each state.
    """
    start_counts = np.zeros(num_states)
    
    # Count occurrences of each state as the starting state
    start_counts[label_indices[0]] += 1
    
    for i in range(1, len(label_indices)):
        if label_indices[i - 1] != label_indices[i]:
            start_counts[label_indices[i]] += 1
    
    # Calculate start probabilities
    total_starts = np.sum(start_counts)
    start_probs = start_counts / total_starts
    
    return start_probs

In [148]:
ES_start_probs = calculate_start_probs(ES_label_idx, len(ES_states))
RU_start_probs = calculate_start_probs(RU_label_idx, len(RU_states))

print("Start Probabilities for ES:", ES_start_probs)
print("Start Probabilities for RU:", RU_start_probs)

Start Probabilities for ES: [0.02094241 0.33653287 0.03926702 0.004363   0.11082024 0.01977894
 0.46829552]
Start Probabilities for RU: [0.0378497  0.33900165 0.06399707 0.00511977 0.08118486 0.0149936
 0.45785336]


## 4.5 Viterbi Algorithm for Best Sequences
The function belows is differnt from the first due to the need of handling np.arrays.

In [230]:
import numpy as np

def viterbi_algorithm(observations, states, start_probs, transition_probs, emission_probs, word_mapping):
    """
    Runs the Viterbi algorithm to find the most likely sequence of hidden states.

    INPUTS:
    - observations (list): List of observed words/tokens.
    - states (list): List of all possible states (sentiment labels).
    - start_probs (np.array): Initial state probabilities.
    - transition_probs (np.array): Transition probabilities matrix (state x state x state).
    - emission_probs (np.array): Emission probabilities matrix (state x word).
    - word_mapping (dict): Mapping of words to index in emissions probabiltiy np array

    OUTPUTS:
    - best_path (list): List of the most likely sequence of hidden states.
    """
    num_states = len(states)
    num_observations = len(observations)
    
    # Create a Viterbi matrix to store probabilities and backpointers
    viterbi_matrix = np.zeros((num_states, num_observations))
    backpointers = np.zeros((num_states, num_observations), dtype=int)
    
    # Initialize the first column of the Viterbi matrix using start probabilities
    viterbi_matrix[:, 0] = start_probs * emission_probs[:, word_mapping[observations[0]]]
    
    # Fill in the Viterbi matrix and backpointers
    for t in range(1, num_observations):
        for s in range(num_states):
            max_prob = -np.inf
            max_prev_state = None
            for prev_state_1 in range(num_states):
                for prev_state_2 in range(num_states):
                    prob = viterbi_matrix[prev_state_2, t - 1] * transition_probs[prev_state_1, prev_state_2, s] * emission_probs[s, word_mapping[observations[t]]]
                    if prob > max_prob:
                        max_prob = prob
                        max_prev_state = prev_state_2
            viterbi_matrix[s, t] = max_prob
            backpointers[s, t] = max_prev_state
    
    # Backtrack to find the best path
    best_path = [0] * num_observations
    best_path[-1] = np.argmax(viterbi_matrix[:, -1])
    for t in range(num_observations - 2, -1, -1):
        best_path[t] = backpointers[best_path[t + 1], t + 1]
    
    return best_path

# Example usage
ES_states_indexed = [i for i in range(len(ES_states))]
RU_states_indexed = [i for i in range(len(RU_states))]
best_path_ES = viterbi_algorithm(ES_texts, ES_states_indexed, ES_start_probs, ES_a, ES_b, ES_wordmapping)
best_path_RU = viterbi_algorithm(RU_texts, RU_states_indexed , RU_start_probs, RU_a, RU_b, RU_wordmapping)

print("Best Path for ES:", best_path_ES)
print("Best Path for RU:", len(best_path_RU))


Best Path for ES: 31086
Best Path for RU: 43838


## 4.6 Model Evaluation
Calculation of F-score, Precision, Recall

In [231]:
RU_state_mappings

{'B-neutral': 0,
 'B-positive': 1,
 'I-positive': 2,
 'I-neutral': 3,
 'B-negative': 4,
 'I-negative': 5,
 'O': 6}

In [232]:
def calculate_metrics(predicted_states, true_states, positive_labels):
    """
    Calculates F-score, Precision, and Recall based on predicted and true state sequences.

    INPUTS:
    - predicted_states (list): Predicted state sequence (indices or labels).
    - true_states (list): True state sequence (indices or labels).
    - positive_label (arr): The positive sentiment labels

    OUTPUTS:
    - precision (float): Precision score.
    - recall (float): Recall score.
    - f_score (float): F-score.
    """
    true_positive = sum(1 for pred, true in zip(predicted_states, true_states) if pred == true and pred in positive_labels)
    false_positive = sum(1 for pred, true in zip(predicted_states, true_states) if pred != true and pred in positive_labels)
    false_negative = sum(1 for pred, true in zip(predicted_states, true_states) if pred != true and pred in positive_labels)
    
    precision = true_positive / (true_positive + false_positive + 1e-10)
    recall = true_positive / (true_positive + false_negative + 1e-10)
    f_score = 2 * (precision * recall) / (precision + recall + 1e-10)
    
    return precision, recall, f_score

In [233]:
positive_labels = [1, 2]
RU_precision, RU_recall, RU_fscore = calculate_metrics(best_path_RU, RU_label_idx, positive_labels)
ES_precision, ES_recall, ES_fscore = calculate_metrics(best_path_ES, ES_label_idx, positive_labels)

In [234]:
print(RU_precision, RU_recall, RU_fscore)
print(ES_precision, ES_recall, ES_fscore)

0.8749999999945314 0.8749999999945314 0.8749999999445314
0.8571428571306122 0.8571428571306122 0.8571428570806122


In [239]:
for i in range(len(best_path_ES)):
    if best_path_ES[i] == ES_label_idx[i]:
        print(f"{i}, {best_path_ES[i]}")

0, 6
1, 6
2, 6
3, 6
4, 6
5, 6
6, 6
7, 6
8, 6
9, 6
10, 6
11, 6
12, 6
13, 6
14, 6
15, 6
16, 1
17, 6
18, 6
20, 6
21, 6
22, 1
23, 6
24, 6
25, 1
26, 6
27, 6
28, 6
29, 6
30, 6
31, 6
32, 6
33, 6
34, 6
35, 6
36, 6
37, 6
38, 6
40, 6
41, 6
42, 6
43, 6
44, 6
45, 6
46, 6
47, 6
48, 6
49, 6
50, 6
51, 6
52, 6
53, 6
54, 6
55, 6
56, 6
57, 6
58, 6
59, 6
60, 1
61, 6
62, 6
64, 6
65, 6
66, 6
67, 6
68, 6
69, 6
70, 6
71, 6
72, 6
73, 6
74, 6
75, 1
76, 6
77, 6
78, 6
79, 6
80, 6
81, 6
82, 6
83, 6
84, 6
85, 6
86, 1
87, 6
88, 6
89, 6
153, 0
591, 0
685, 0
1053, 0
1180, 0
1467, 0
1973, 0
1994, 0
3110, 0
3679, 0
4170, 0
5573, 0
6337, 0
6727, 0
7076, 0
7621, 0
7639, 0
8725, 0
9741, 0
9876, 0
9937, 0
10650, 0
11215, 0
12060, 0
12898, 0
13053, 0
13203, 0
13284, 0
13817, 0
14263, 0
14309, 0
15112, 0
15116, 0
15126, 0
15130, 0
15316, 0
16647, 0
17242, 0
17836, 0
18232, 0
18263, 0
18367, 0
18779, 0
19226, 0
19758, 0
20155, 0
20158, 0
22200, 0
22643, 0
22875, 0
23062, 0
23230, 0
23714, 0
23891, 0
24156, 0
24555, 0
24991, 0